# Simple Trend-Following strategy

## Background

Applying a Simple Trend-Following strategy, where trades are made based on the direction of recent price movements. In this specific case, the strategy is based on the percentage change in the OHLC (open, high, low, close) prices, with a buy signal generated if the percentage change is positive and a sell signal generated if the percentage change is negative.

---
## 0. Imports

In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.preprocessing import StandardScaler

import sys
sys.path.append('../functions_library')

from functions import ROC, model_selection, Backtesting

#import sklearn 
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
import sklearn.ensemble

# Classification Metrics 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc #plot_roc_curve, auc
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# ML models 
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import xgboost as xgb

# libraries for Shapely analysis
import shap 

seed = 42

---
## 1. Data Loading

Read the CSV file into Pandas DataFrame

In [2]:
# Import the OHLCV dataset into a Pandas Dataframe
trading_df = pd.read_csv(
    Path("../Resources/ETHUSDT-1h-data.csv"), 
    index_col="timestamp", 
    infer_datetime_format=True, 
    parse_dates=True
)

trading_df = trading_df.drop(["close_time","quote_av","trades","tb_base_av","tb_quote_av","ignore"], axis =1)

# Review the DataFrame
trading_df.head()

,open,high,low,close,volume
timestamp,,,,,
2017-08-17 04:00:00,301.13,302.57,298.0,301.61,125.66877
2017-08-17 05:00:00,301.61,303.28,300.0,303.10,377.67246
2017-08-17 06:00:00,302.40,304.44,301.9,302.68,303.86672
2017-08-17 07:00:00,302.68,307.96,302.6,307.96,754.74510
2017-08-17 08:00:00,307.95,309.97,307.0,308.62,150.75029


Add a daily return values column to the DataFrame

In [3]:
# Calculate the daily returns using the closing prices and the pct_change function
trading_df["actual_returns"] = trading_df["close"].pct_change()

# Drop all NaN values from the DataFrame
trading_df = trading_df.dropna()

# Review the DataFrame
display(trading_df.head())
display(trading_df.tail())

,open,high,low,close,volume,actual_returns
timestamp,,,,,,
2017-08-17 05:00:00,301.61,303.28,300.00,303.10,377.67246,0.004940
2017-08-17 06:00:00,302.40,304.44,301.90,302.68,303.86672,-0.001386
2017-08-17 07:00:00,302.68,307.96,302.60,307.96,754.74510,0.017444
2017-08-17 08:00:00,307.95,309.97,307.00,308.62,150.75029,0.002143
2017-08-17 09:00:00,308.62,312.00,308.62,310.00,469.27879,0.004472


,open,high,low,close,volume,actual_returns
timestamp,,,,,,
2023-01-23 18:00:00,1629.03,1640.49,1626.06,1628.85,26815.5440,-0.000104
2023-01-23 19:00:00,1628.85,1630.57,1610.04,1619.88,27357.8436,-0.005507
2023-01-23 20:00:00,1619.89,1635.26,1618.65,1634.80,13128.4739,0.009211
2023-01-23 21:00:00,1634.80,1635.50,1629.53,1631.84,9298.6748,-0.001811
2023-01-23 22:00:00,1631.84,1635.75,1630.90,1632.64,5790.1341,0.000490


---
## 2. Feature Engineering

In [4]:
import finta as ft
from finta import TA
import talib

ohlcv_df = trading_df

# List of time periods to use for Moving Averages calculation
timeperiods = [5,7,14,20,30,50,70,100,150,200]

df = ohlcv_df.copy()

# Calculate SMAs and add them to the DataFrame
for t in timeperiods:
    #tsma = TA.SMA(df, t).shift(1)
    sma = TA.SMA(df, t)
    ema = TA.EMA(df, t)
    atr = TA.ATR(df, t)  #Average True Range
    adx = TA.ADX(df, t) 
    rsi = TA.RSI(df, t)
    hma = TA.HMA(df, t)
    vama = TA.VAMA(df, t)
 
    # calculate the Force Index
    force_index = pd.Series(df['close'].diff(1) * df['volume'], index=df.index)
    force_ema = force_index.ewm(span=t, min_periods=0, adjust=True, ignore_na=False).mean()    
    
    #df['force_index'] = force_index
    #df[f'force_index_ema_{t}'] = force_ema # add the Force Index and its EMA to the DataFrame
    #df[f'TSMA_{t}'] = tsma
    df[f'SMA_{t}'] = sma
    df[f'EMA_{t}'] = ema
    df[f'HMA_{t}'] = hma
    df[f'VAMA_{t}'] = vama
    df[f'ATR_{t}'] = atr
    df[f'ADX_{t}'] = adx
    df[f'RSI_{t}'] = rsi
    
    
# Calculate the Parabolic SAR
#sar = TA.PSAR(df)

# Add the SAR values and trend direction to the DataFrame
#df['sar'] = sar['psar']
#df['psarbear'] = sar['psarbear']
#df['psarbull'] = sar['psarbull']

df['UO'] = TA.UO(df)

# Adding Awesome Indicator (AO)
df['AO'] = TA.AO(df)
df['OBV'] =TA.OBV(df)

# Adding Chaikin Indicator 
df['CHAIKIN'] = TA.CHAIKIN(df)

# Adding Bollinger Bands
df[['BB_UPPER','BB_MED','BB_LOWER']] =TA.BBANDS(df)

# Calculate the Keltner Channel with TALIB
#df[['KC_UPPER','KC_MED','KC_LOWER']] = TA.KC(df)

# calculate Commodity Channel Index (CCI)
df['cci'] = TA.CCI(df)

# assuming you have OHLCV data in a pandas dataframe called "df"
#volume_momentum = talib.MOM(df['volume'])

In [5]:

# calculate the Ichimoku Kinko Hyo indicator
# Calculate the conversion line
nine_period_high = df['high'].rolling(window=9).max()
nine_period_low = df['low'].rolling(window=9).min()
df['tenkan_sen'] = (nine_period_high + nine_period_low) / 2

# Calculate the base line
periods = 26
twenty_six_period_high = df['high'].rolling(window=periods).max()
twenty_six_period_low = df['low'].rolling(window=periods).min()
df['kijun_sen'] = (twenty_six_period_high + twenty_six_period_low) / 2

# Calculate the leading span A
df['senkou_span_a'] = ((df['tenkan_sen'] + df['kijun_sen']) / 2).shift(periods=periods)

# Calculate the leading span B
periods2 = 52
fifty_two_period_high = df['high'].rolling(window=periods2).max()
fifty_two_period_low = df['low'].rolling(window=periods2).min()
df['senkou_span_b'] = ((fifty_two_period_high + fifty_two_period_low) / 2).shift(periods=periods)

# Calculate the lagging span
df['chikou_span'] = df['close'].shift(periods=-periods)


#### Add a daily return values column to the DataFrame

In [6]:
# Calculate the daily returns using the closing prices and the pct_change function
df["actual_returns"] = df["close"].pct_change()

# Drop all NaN values from the DataFrame
df = df.dropna()

# Review the DataFrame
df.head()

,open,high,low,close,volume,actual_returns,SMA_5,EMA_5,HMA_5,VAMA_5,...,CHAIKIN,BB_UPPER,BB_MED,BB_LOWER,cci,tenkan_sen,kijun_sen,senkou_span_a,senkou_span_b,chikou_span
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-09-02 19:00:00,340.23,341.47,334.93,336.18,234.99340,-0.009166,339.158,339.736879,337.449778,339.376015,...,-137.517059,393.172933,357.5685,321.964067,-102.539757,343.685,360.705,388.9250,384.305,341.71
2017-09-02 20:00:00,338.08,338.47,325.63,325.63,452.29677,-0.031382,335.940,335.034586,328.105556,334.957058,...,-282.506411,388.022171,354.1340,320.245829,-133.249723,342.955,359.975,389.2000,385.245,341.38
2017-09-02 21:00:00,327.16,331.08,320.08,324.00,236.00212,-0.005006,333.346,331.356391,320.744222,332.311002,...,-337.709600,384.387306,351.1105,317.833694,-139.374949,338.830,357.200,389.2000,386.085,341.77
2017-09-02 22:00:00,325.45,340.10,325.45,329.72,229.42859,0.017654,330.964,330.810927,324.167778,329.932132,...,-360.037586,379.516187,348.4625,317.408813,-93.896395,337.920,357.200,389.3675,386.085,333.09
2017-09-02 23:00:00,331.77,343.15,329.72,343.14,280.21839,0.040701,331.734,334.920618,338.516000,331.607894,...,-247.413454,374.386794,346.6935,319.000206,-50.887796,332.675,357.200,389.3675,386.085,335.65


In [7]:
# Create a new column in the trading_df called signal setting its value to zero.
df["signal"] = 0.0

# Create the signal to buy
df.loc[(df["actual_returns"] >= 0), "signal"] = 1

# Create the signal to sell
df.loc[(df["actual_returns"] < 0), "signal"] = -1

In [8]:
df["signal"].value_counts()

 1.0    23777
-1.0    23206
Name: signal, dtype: int64

---
## 3. Pre-Processing
### 3.1 Dealing with Class Imbalance using Undersampling 

In [9]:
import pandas as pd
from sklearn.utils import resample

# Count the number of samples in each class
class_counts = df['signal'].value_counts()

# Find the class with fewer samples
minority_class = class_counts.idxmin()

# Split the dataframe into the majority and minority classes
majority_class = df[df['signal'] != minority_class]
minority_class = df[df['signal'] == minority_class]

# Undersample the majority class to match the number of samples in the minority class
undersampled_majority = resample(majority_class,
                                 replace=False,
                                 n_samples=len(minority_class),
                                 random_state=42)

# Combine the undersampled majority class with the minority class
balanced_df = pd.concat([undersampled_majority, minority_class])

# Shuffle the rows in the balanced dataframe
balanced_df = balanced_df.sample(frac=1, random_state=42)

> The class imbalance method (undersampling) was applied, but it did not improve the results significantly, particularly for the Test dataset's ROC curve. Uncommenting the line below will allow for the option to skip the undersampling method for the remainder of the process.

In [10]:
balanced_df = df #comment this line if you want to use balanced classes

In [11]:
balanced_df["signal"].value_counts()

 1.0    23777
-1.0    23206
Name: signal, dtype: int64

------
### 3.2 Generating the Feature and Target datasets

#### Creating the feature set

In [12]:
# Creating X as predictors (features) dataset
X= balanced_df.copy()

# Removing columns indicative of previous prices
# & shifting X one row down to avoid using data not available to us at the time of prediction
X= X.drop(["open","high","low","close","volume","actual_returns","signal"], axis =1).shift().dropna().copy()

In [13]:
X.head()

,SMA_5,EMA_5,HMA_5,VAMA_5,ATR_5,ADX_5,RSI_5,SMA_7,EMA_7,HMA_7,...,CHAIKIN,BB_UPPER,BB_MED,BB_LOWER,cci,tenkan_sen,kijun_sen,senkou_span_a,senkou_span_b,chikou_span
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-09-02 20:00:00,339.158,339.736879,337.449778,339.376015,11.534,77.693325,31.151460,340.600000,341.680162,337.664603,...,-137.517059,393.172933,357.5685,321.964067,-102.539757,343.685,360.705,388.9250,384.305,341.71
2017-09-02 21:00:00,335.940,335.034586,328.105556,334.957058,10.518,79.661445,20.476812,336.677143,337.667621,330.740873,...,-282.506411,388.022171,354.1340,320.245829,-133.249723,342.955,359.975,389.2000,385.245,341.38
2017-09-02 22:00:00,333.346,331.356391,320.744222,332.311002,10.104,81.864543,19.205792,335.060000,334.250716,323.166786,...,-337.709600,384.387306,351.1105,317.833694,-139.374949,338.830,357.200,389.2000,386.085,341.77
2017-09-02 23:00:00,330.964,330.810927,324.167778,329.932132,10.858,71.317552,36.496297,333.345714,333.118037,322.624325,...,-360.037586,379.516187,348.4625,317.408813,-93.896395,337.920,357.200,389.3675,386.085,333.09
2017-09-03 00:00:00,331.734,334.920618,338.516000,331.607894,11.982,60.038337,60.983606,334.227143,335.623528,332.935833,...,-247.413454,374.386794,346.6935,319.000206,-50.887796,332.675,357.200,389.3675,386.085,335.65


#### Creating the target set 

In [14]:
# Copy the new signal column to a new Series called y.
y = balanced_df["signal"].copy()

# keeping y and X the same size 
y = y[X.index]

#### QC of X,y shape and content

In [15]:
X.shape

(46982, 83)

In [16]:
y.shape

(46982,)

In [17]:
y.value_counts()

 1.0    23777
-1.0    23205
Name: signal, dtype: int64

In [18]:
X.columns

Index(['SMA_5', 'EMA_5', 'HMA_5', 'VAMA_5', 'ATR_5', 'ADX_5', 'RSI_5', 'SMA_7',
       'EMA_7', 'HMA_7', 'VAMA_7', 'ATR_7', 'ADX_7', 'RSI_7', 'SMA_14',
       'EMA_14', 'HMA_14', 'VAMA_14', 'ATR_14', 'ADX_14', 'RSI_14', 'SMA_20',
       'EMA_20', 'HMA_20', 'VAMA_20', 'ATR_20', 'ADX_20', 'RSI_20', 'SMA_30',
       'EMA_30', 'HMA_30', 'VAMA_30', 'ATR_30', 'ADX_30', 'RSI_30', 'SMA_50',
       'EMA_50', 'HMA_50', 'VAMA_50', 'ATR_50', 'ADX_50', 'RSI_50', 'SMA_70',
       'EMA_70', 'HMA_70', 'VAMA_70', 'ATR_70', 'ADX_70', 'RSI_70', 'SMA_100',
       'EMA_100', 'HMA_100', 'VAMA_100', 'ATR_100', 'ADX_100', 'RSI_100',
       'SMA_150', 'EMA_150', 'HMA_150', 'VAMA_150', 'ATR_150', 'ADX_150',
       'RSI_150', 'SMA_200', 'EMA_200', 'HMA_200', 'VAMA_200', 'ATR_200',
       'ADX_200', 'RSI_200', 'UO', 'AO', 'OBV', 'CHAIKIN', 'BB_UPPER',
       'BB_MED', 'BB_LOWER', 'cci', 'tenkan_sen', 'kijun_sen', 'senkou_span_a',
       'senkou_span_b', 'chikou_span'],
      dtype='object')

--- 
### 3.3 Feature Selection 
####  Use RFE (Recursive Feature Elimination) to keep the best features, with logistic regression as base model

In [ ]:
# Create a logistic regression model
model = LogisticRegression(max_iter=1500)

# Create an RFE model to select the best features
rfe = RFE(model, n_features_to_select=15)

# Fit the RFE model to the data
rfe = rfe.fit(X, y)

# Get the selected features
selected_features = X.columns[rfe.support_]

# keep only specified columns in the dataframe
#X = X.iloc[:, selected_features]
X = X[selected_features]

# Print the selected features
print ("Number of selected features:", len(X.columns), "\nSelected features:\n",selected_features)

In [ ]:
%%capture
''' keeping note of the best selected features: 
Selected features:
 Index(['EMA_5', 'HMA_5', 'RSI_5', 'HMA_14', 'VAMA_20', 'RSI_30', 'HMA_50',
       'ATR_50', 'RSI_50', 'HMA_70', 'VAMA_70', 'RSI_70', 'RSI_100',
       'tenkan_sen', 'chikou_span'],
      dtype='object')
'''

#### Remove highly-correlated features

In [ ]:
df_feats = X.copy()
# Calculate correlation matrix
# calculate the correlation matrix of the features
corr_matrix = df_feats.corr()

# set the threshold for correlation value
corr_threshold = 0.90

# find the highly correlated features and drop them from the dataframe
high_corr_features = np.where(corr_matrix.abs() > corr_threshold)
high_corr_features = [(corr_matrix.columns[x], corr_matrix.columns[y]) for x, y in zip(*high_corr_features) if x != y and x < y]
df_feats.drop([col[1] for col in high_corr_features], axis=1, inplace=True)

# select the remaining features with low correlation
low_corr_features = df_feats.columns.tolist()

# print the low correlated features
print(low_corr_features)

> The removal of the highly correlated feature does not appear to improve the quality of the prediction. Commenting out the line below will result in keeping only the features with less correlation.

In [ ]:
#X = X[low_corr_features]

---
### 3.4 Splitting data and creating train and test datasets using time offset

In [ ]:
# Imports 
from pandas.tseries.offsets import DateOffset

# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
#print(training_begin)

# Select the ending period for the training data with an offset of 36 months
training_end = X.index.min() + DateOffset(months=24)

# Display the training end date
#print(training_end)

# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Display sample data
display(X_train.head())

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Display sample data
display(X_test.head())

___
### 3.5 Standardizing the Data

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---
## 4. ML models 
#### 4.1 XGboost

In [ ]:
import xgboost as xgb
#from xgboost import xgbClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV

#replace -1 by 0 as xgboost expects a boolean target vector (only 0 and 1)
y_train = y_train.replace(-1, 0)
y_test = y_test.replace(-1, 0)

# Create a xgb Classifier model
xgb_clf = xgb.XGBClassifier(reg_alpha=27)

xgb_clf.fit(X_train_scaled, y_train) 

# Use the best model to make predictions on the test data
y_pred_xgb = xgb_clf.predict(X_test_scaled)

ROC(xgb_clf,X_train,X_test,y_train,y_test)

In [ ]:
# Define the trading fee as a decimal
trading_fee = 0.00075
classifier = xgb_clf

# Backtest using our exteranl function
xgb_predictions_df = Backtesting (df, X_test, X_test_scaled, classifier,trading_fee)

# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
(1 + xgb_predictions_df[["actual_returns", "trading_algorithm_returns"]]).cumprod().plot()

In [ ]:
X_test.columns


---
#### 4.2 CatBoost

In [ ]:
# Import libraries
from catboost import CatBoostClassifier
# Initialize the CatBoost model

cb_clf = CatBoostClassifier(iterations=450,random_state=seed  ,l2_leaf_reg=55.8, verbose = False)

# Train the model on the training data
cb_clf.fit(X_train_scaled, y_train)

# display ROC and classification metrics 
ROC(cb_clf,X_train_scaled, X_test_scaled, y_train, y_test)


In [ ]:
# Define the trading fee as a decimal
trading_fee = 0.00075
classifier = cb_clf

# Backtest using our exteranl function
cb_predictions_df = Backtesting (df, X_test, X_test_scaled, classifier,trading_fee)

# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
(1 + cb_predictions_df[["actual_returns", "trading_algorithm_returns"]]).cumprod().plot()

---
#### 4.3 LDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# create LDA object and fit the model
lda = LinearDiscriminantAnalysis(n_components=1)

# Train the model on the training data
lda.fit_transform(X_train_scaled, y_train)

ROC(lda,X_train_scaled, X_test_scaled, y_train, y_test)

In [ ]:
# Define the trading fee as a decimal
trading_fee = 0.00075
classifier = lda

# Backtest using our exteranl function
lda_predictions_df = Backtesting (df, X_test, X_test_scaled, classifier,trading_fee)

# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
(1 + lda_predictions_df[["actual_returns", "trading_algorithm_returns"]]).cumprod().plot()

> LDA model seems way too good to be true. I'm not sure whether I'm using properly this algorithm or not. I still need to research more about it. This behavior makes me doubt about the quality of my selected features, as it looks like I'm using some data here for prediction that I don't have access to (data from the future) which can lead to an anormaly good behavior. 

---
### 4.4 Model Building helpers

These utilities are helpful for identifying the best models and understanding them better. However, they may take a long time to run. If needed, you can interrupt their execution as they are not essential for the machine learning process.

### Selecting best performing model 

In [ ]:
model_selection(X_test,y_test)

### SHAP Analysis 

In [ ]:
# Resample X_test
upscaled_X_test = X_test.resample('1D').interpolate(method='linear')


In [ ]:
# define which classifier to use 
clf = xgb_clf


# Initialize SHAP explainer
explainer = shap.Explainer(clf.predict, upscaled_X_test)

# Calculate SHAP values
shap_values = explainer(upscaled_X_test)

shap.summary_plot(shap_values, upscaled_X_test, plot_type="bar")

In [ ]:
# SHAP Analysis 
# define which classifier to use 
clf = cb_clf

# Initialize SHAP explainer
explainer = shap.Explainer(clf.predict, upscaled_X_test)

# Calculate SHAP values
shap_values = explainer(upscaled_X_test)

shap.summary_plot(shap_values, upscaled_X_test, plot_type="bar")